In [1]:
# Celda 1: importar librerías
import requests
import pandas as pd
import time
import json
from sqlalchemy import create_engine, text
from time import sleep

tipos_list = []
habilidades_list = []
lista_poke = []

In [2]:
usuario = "postgres"         # o el que hayas definido
password = "1234"  # la que pusiste al instalar PostgreSQL
host = "localhost"
puerto = "5432"
nombre_bd = "pokemon_db"

engine = create_engine(f'postgresql+psycopg2://{usuario}:{password}@{host}:{puerto}/{nombre_bd}')

with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS pokemon"))
    conn.commit()  # <-- esto es lo que probablemente faltaba

In [3]:
resp = requests.get('https://pokeapi.co/api/v2/pokemon')
data = resp.json()
total_pokemons = data['count']

In [4]:
def extrae_pokemons(id):
    try:
        resp = requests.get(f'https://pokeapi.co/api/v2/pokemon/{id}')
        time.sleep(0.5)
        if resp.status_code != 200:
            print(f"Fallo en el id {id}")
            return None
        data = resp.json()

         # Stats como diccionario
        stats = {s["stat"]["name"]: s["base_stat"] for s in data["stats"]}

        # Tipos y habilidades como listas
        tipos = [t["type"]["name"] for t in data["types"]]
        habilidades = [a["ability"]["name"] for a in data["abilities"]]

        # También llenamos las tablas relacionales si las vas a usar luego
        for t in tipos:
            tipos_list.append({"pokemon_id": data["id"], "tipo": t})
        for h in habilidades:
            habilidades_list.append({"pokemon_id": data["id"], "habilidad": h})

        return{
            "id": data["id"],
            "name": data["name"],
            "height": data["height"],
            "weight": data["weight"],
            "base_experience": data["base_experience"],
            "types": json.dumps(tipos),       # Guardado como texto
            "abilities": json.dumps(habilidades),
            "hp": stats.get("hp"),
            "attack": stats.get("attack"),
            "defense": stats.get("defense")
        }
    except Exception as e:
        print(f"error con id{id}: {e}")
        return None


In [5]:
for i in range(1, total_pokemons, 100):
    grupo = list(range(i, min(i + 100, total_pokemons)))

    for poke_id in grupo:
        info = extrae_pokemons(poke_id)  # <--- aquí defines `info`
        if info:                         # <--- validas que no sea None
            lista_poke.append(info)

    df = pd.DataFrame(lista_poke)
    df.to_sql('pokemon', engine, if_exists='append', index=False)
    lista_poke = []
    sleep(1)

Fallo en el id 1026
Fallo en el id 1027
Fallo en el id 1028
Fallo en el id 1029
Fallo en el id 1030
Fallo en el id 1031
Fallo en el id 1032
Fallo en el id 1033
Fallo en el id 1034
Fallo en el id 1035
Fallo en el id 1036
Fallo en el id 1037
Fallo en el id 1038
Fallo en el id 1039
Fallo en el id 1040
Fallo en el id 1041
Fallo en el id 1042
Fallo en el id 1043
Fallo en el id 1044
Fallo en el id 1045
Fallo en el id 1046
Fallo en el id 1047
Fallo en el id 1048
Fallo en el id 1049
Fallo en el id 1050
Fallo en el id 1051
Fallo en el id 1052
Fallo en el id 1053
Fallo en el id 1054
Fallo en el id 1055
Fallo en el id 1056
Fallo en el id 1057
Fallo en el id 1058
Fallo en el id 1059
Fallo en el id 1060
Fallo en el id 1061
Fallo en el id 1062
Fallo en el id 1063
Fallo en el id 1064
Fallo en el id 1065
Fallo en el id 1066
Fallo en el id 1067
Fallo en el id 1068
Fallo en el id 1069
Fallo en el id 1070
Fallo en el id 1071
Fallo en el id 1072
Fallo en el id 1073
Fallo en el id 1074
Fallo en el id 1075
